In [1]:
import xml.etree.ElementTree as ET
from FAIRFlowChemistry.core import PlantSetup
from FAIRFlowChemistry.core import Component
from FAIRFlowChemistry.core import ComponentType
from typing import DefaultDict
from pathlib import Path


def DEXPI2sdRDM(filepath: Path):
    tree = ET.parse(filepath)
    root = tree.getroot()


    def build_component(component, component_type: ComponentType):
        return Component(
            component_type=component_type,
            component_id=component.get('ID'),
            component_class=component.get('ComponentClass'),
            component_class_uri=component.get('ComponentClassURI')
        )

    components = []
    nozzles = DefaultDict(list)
    for eq in root.findall('Equipment'):
        components.append(build_component(eq, ComponentType.EQUIPMENT))
        for nozzle in eq.findall('Nozzle'):
            nozzles[eq.get('ID')].append(nozzle.get('ID'))

    connections = []
    piping_network_system = root.findall('PipingNetworkSystem')
    for pns in piping_network_system:
        for piping_network_segment in pns.findall('PipingNetworkSegment'):
            connection = [piping_network_segment.find('Connection').get('FromID'), piping_network_segment.find('Connection').get('ToID')]
            connections.append(connection)
            for piping_component in piping_network_segment.findall('PipingComponent'):
                components.append(build_component(piping_component, ComponentType.PIPINGCOMPONENT))
  
    for connection in connections:
        for connection_point in connection:
            if 'Nozzle' in connection_point:
                for eq, nozzle in nozzles.items():
                    for n in nozzle:
                        if n in connection_point:
                            connection[connection.index(connection_point)] = eq
    for component in components:
        for connection in connections:
            if component.component_id in connection:
                component.connections.extend(connection)
                component.connections.remove(component.component_id)

    return PlantSetup(components=components)

cwd = Path.cwd()
filepath = cwd / "data" / "DEXPI" / "DEXPI_PID.xml"
plant_setup = DEXPI2sdRDM(filepath)

plant_setup

/home/shambles/miniconda3/envs/b07/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


PlantSetup(id='374f72ed-7f50-4346-8714-dfbef6b16521', components=[Component(id='9a9f0164-b31f-4ed5-8ae1-4f87f8aa7390', component_type='Equipment', component_id='GasCylinder-1', component_class='GasCylinder', component_class_uri='https://data.posccaesar.org/rdl/RDS4316755815', component_name=None, generic_attributes=[], connections=['Valve-1']), Component(id='1e9dc67e-a224-451a-977f-22406806b45d', component_type='Equipment', component_id='GasCylinder-2', component_class='GasCylinder', component_class_uri='https://data.posccaesar.org/rdl/RDS4316755815', component_name=None, generic_attributes=[], connections=['Valve-1']), Component(id='45363515-8d79-40a0-9f45-f13a6d37f44b', component_type='Equipment', component_id='Vessel-1', component_class='Vessel', component_class_uri='https://data.posccaesar.org/rdl/RDS414674', component_name=None, generic_attributes=[], connections=['Valve-1', 'Valve-2']), Component(id='27729983-d2ab-40fe-922b-1e64166008ac', component_type='Equipment', component_id=

Notes: Valves and similar components belong to the PipingNetworkSystem domain; they do not represent an independent category. Moreover, they are subordinated to a PipingNetworkSegment. More specifically to the one, that starts from this Component.